In [1]:
from sklearn.utils import class_weight, shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.inspection import permutation_importance

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from xgboost import XGBRegressor, XGBClassifier

import scipy.stats as stats
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
BATCH_SIZE = 32
EPOCHS = 300

from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
import cx_Oracle
import pandas as pd
from sqlalchemy import create_engine
from imblearn.over_sampling import SMOTE
from collections import Counter
import joblib
import shap

class Aggregate_helper:
    def __init__(self, eq_id, lookback_window, major_down_hour, alarm_table):
        self.eq_id = eq_id
        self.lookback_window = lookback_window
        self.alarm_table = alarm_table
        self.status_table = self.query_status()
        self.major_down_hour = major_down_hour
        
        ## include all status instead since further preprocessing would be performed
        ## make sure that the timeframe table is a subset of both the alarm and status table to compute major down correctly
        time1 = self.alarm_table.iloc[0]["DT_SET"]
        time2 = self.status_table.iloc[0]["TIMESTAMP_START"]
        timeend1 = self.alarm_table.iloc[len(self.alarm_table)-1]["DT_SET"]
        timeend2 = self.status_table.iloc[len(self.status_table)-1]["TIMESTAMP_START"]
        
        # give a 3 days window to ensure that the alarm and status are captured fully
        start = (max(time1, time2) + timedelta(days=3)).strftime("%d/%m/%Y") 
        end = min(timeend1, timeend2).strftime("%d/%m/%Y")
        
        self.timeframe_table = self.generate_time(start, end, 3)
        self.major_down_arr = self.major_down(self.timeframe_table, self.status_table, self.major_down_hour, 3600)
        self.aggregated = self.aggregate(self.timeframe_table, self.lookback_window, self.alarm_table, self.status_table)
        self.aggregated_table = pd.concat([self.timeframe_table.reset_index(drop=True), self.aggregated.reset_index(drop=True)], axis=1)
        
    def generate_time(self, start_date:str, end_date:str, hour:int):
        start = datetime.strptime(start_date, '%d/%m/%Y')
        end = datetime.strptime(end_date, '%d/%m/%Y')

        dates = []
        while start<=end:
            row = [start]
            dates.append(row)
            start += timedelta(hours=hour)

        return pd.DataFrame(dates, columns=['TIMESTAMP'])
    
    def query_status(self):
        try:
            oracle_string = "oracle+cx_oracle://{username}:{password}@{hostname}:{port}/{database}"
            engine = create_engine(
                oracle_string.format(
                    username = 'TFM4CEBERUS',
                    password = 'TFM4CEBERUS',
                    hostname = 'ome-db.bth.infineon.com',
                    port = '1538',
                    database = 'ome'
                    )
                )
        except Exception as e:
            print(str(e))

        query = f"""select EQ_ID, TIMESTAMP_START, TIMESTAMP_END, DURATION, STATE_NAME, LEVEL3_NAME, LEVEL3 
                from (SELECT
                  eq.eq_id, eq.name, eq.eq_type_ident
                , data.timestamp_start,data.timestamp_end
                , ROUND((data.timestamp_end - data.timestamp_start)*24*60*60,0) AS Duration
                , data.tr25_3_status,data.tr25_4_status,data.tr25_5_status,data.eq_status
                , level5s.state_name
                , level5.state_name Level5_Name, level5.state_sign Level5
                , level4.state_name Level4_Name, level4.state_sign Level4
                , level3.state_name Level3_Name, level3.state_sign Level3
                ,mh.device
                ,mh.package,
                mh.lotid as lot,
                mh.product,
                mh.operation

                FROM OMEDATA.EQUIPMENT_STATE_HISTORY data
                , OMEADMIN.EQUIPMENT_INSTANCES eq
                , V_EQ_STATES level5s
                , OMEADMIN.DEF_STANDARD_STATEMODEL level5
                , OMEADMIN.DEF_STANDARD_STATEMODEL level4
                , OMEADMIN.DEF_STANDARD_STATEMODEL level3
                , OMEDATA.METAKEY_HISTORY mh

                WHERE data.eq_ident  = eq.eq_ident
                AND  data.eq_status = level5s.state_ident(+)
                AND level5.state_ident = data.tr25_5_status
                AND level4.state_ident = data.tr25_4_status
                AND level3.state_ident = data.tr25_3_status
                AND  data.metakey_ident =mh.ident(+)
                and data.timestamp_start > sysdate - 1050)
                where eq_id = '{self.eq_id}'
                ORDER BY TIMESTAMP_START"""

        status = pd.read_sql(query, engine)
        status.columns = map(lambda x: str(x).upper(), status.columns) 

        return status
    
    def aggregate(self, timeframe_table, lookback_window, alarm_table, status_table):
        alarm_df = pd.DataFrame()
        statename_df = pd.DataFrame()

        for idx, row in timeframe_table.iterrows():
            end = row["TIMESTAMP"]
            start = end - timedelta(hours=lookback_window)

            ## count the frequencies of each alarm
            filtered_alarm = alarm_table.loc[(alarm_table["DT_SET"] >= start) & (alarm_table["DT_SET"] <= end)]
            alarm_freq_table = filtered_alarm["Alarm ID"].value_counts().to_frame().T.reset_index(drop=True)
            alarm_df = pd.concat([alarm_df, alarm_freq_table], axis=0)

            ## count the frequencies of each statename, include everything since feature engineering would be performed
            filtered_statename = status_table.loc[(status_table["TIMESTAMP_START"] >= start) & (status_table["TIMESTAMP_START"] <= end)]
            status_freq_table = filtered_statename["STATE_NAME"].value_counts().to_frame().T.reset_index(drop=True)
            statename_df = pd.concat([statename_df, status_freq_table], axis=0)
        
        df = pd.concat([alarm_df, statename_df], axis=1) # current dataframe
        df = df.fillna(0)

        ## convert all columns from float to int
        cols = df.columns
        df[cols] = df[cols].astype(int)
        df["EQUIPMENT"] = self.eq_id
        return df
        
    def major_down(self, input_df, status_table, hour, threshold): 
        hour = pd.Timedelta(hours=hour)
        major_down = []

        for idx, row in input_df.iterrows():
            start = row['TIMESTAMP']
            end = start+hour
            frame = status_table[(status_table['TIMESTAMP_START']>start) & (status_table['TIMESTAMP_START']<end)]
            UD = frame.loc[frame['LEVEL3']=='UDT']

            if len(UD) == 0: #no record within this 6 hours:
                major_down.append(0)
            else:
                time_diff = (UD['TIMESTAMP_END']-UD['TIMESTAMP_START']).dt.seconds
                if any(time_diff>=threshold): #threshold = 3600s
                    major_down.append(1)
                else:
                    major_down.append(0)
        return np.array(major_down)

In [36]:
import os 

all_tables = []
all_target = []

start = datetime.now()
for _, path, filename in os.walk("Data"):
    for ele in filename:
        alarm_file = ele
        eq_id = ele[:6]
        print(eq_id)

        full_alarm = pd.read_excel(f"Data/{alarm_file}", engine='openpyxl', usecols = "B,C,D,F,M")
        equipment = Aggregate_helper(eq_id, 24, 24, full_alarm)

        all_tables.append(equipment.aggregated_table)
        all_target.append(equipment.major_down_arr)

end = datetime.now()
print(f"Data collection took {(end-start).seconds} seconds")

WBA120
WBA121
WBA122
WBA123
WBA124
WBA125
WBA126
WBA127
WBA128
WBA129
WBA130
WBA132
WBA133
WBA134
WBA135
WBA137
Data collection took 1980 seconds


In [37]:
new_training = pd.concat(all_tables, axis=0)
new_training = new_training.fillna(0)

## convert all columns from float to int
df_float = new_training.select_dtypes(include=[np.float])
cols = df_float.columns
new_training[cols] = new_training[cols].astype(int)

new_target = np.concatenate(all_target)

col_order = new_training.columns

In [42]:
new_training

,TIMESTAMP,2,3,4,5,8,10,11,12,13,...,238,Conversion,64,70,236,91,159,131,132,TARGET
0,2021-02-12 00:00:00,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-02-12 03:00:00,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-02-12 06:00:00,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-02-12 09:00:00,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-02-12 12:00:00,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7908,2021-08-26 12:00:00,0,0,0,0,1,0,6,0,0,...,0,0,0,0,0,0,0,0,0,1
7909,2021-08-26 15:00:00,0,0,0,0,1,0,5,0,0,...,0,0,0,0,0,0,0,0,0,1
7910,2021-08-26 18:00:00,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,1
7911,2021-08-26 21:00:00,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
new_training["TARGET"] = new_target

In [31]:
training_df = new_training.drop(["TIMESTAMP", "EQUIPMENT", "TARGET"], axis=1)
X_train, X_val, y_train, y_val = train_test_split(training_df.values, new_target, test_size=0.2, stratify=new_target)
print(Counter(y_train), Counter(y_val))

# time interval of 3 hours (rolling time frame)
# 24 hours: 93% testing accuracy
# 168 hours: 95% RF 92% DT

# time interval of 24 hours (rolling time frame)
# 24 hours: 71% testing accuracy
# 168 hours: 65% accuracy

Counter({0: 7388, 1: 2796}) Counter({0: 1847, 1: 699})


# Check Prediction if train_val_split by index

In [39]:
training_df = new_training.drop(["TIMESTAMP", "EQUIPMENT"], axis=1)
train_pct = int(0.8*len(training_df))
train = training_df[:train_pct]
val = training_df[train_pct:]

X_train = train.drop(["TARGET"], axis=1).values
y_train = train["TARGET"].values

X_val = val.drop(["TARGET"], axis=1).values
y_val = val["TARGET"].values
print(Counter(y_train), Counter(y_val))

# 3 hours interval (rolling time frame)
# 

Counter({0: 58260, 1: 23122}) Counter({0: 15734, 1: 4612})


# Check prediction results when test set is collected from the latest dates

In [ ]:
# collect test set by date
aug = datetime(2021, 8, 1)
test_set = new_training[new_training["TIMESTAMP"]>=aug].drop(["TIMESTAMP", "EQUIPMENT"], axis=1)
training_set = new_training[new_training["TIMESTAMP"]<aug].drop(["TIMESTAMP", "EQUIPMENT"], axis=1)

X_train = training_set.drop(["TARGET"], axis=1).values
y_train = np.array(training_set["TARGET"].values)

X_val = test_set.drop(["TARGET"], axis=1).values
y_val = np.array(test_set["TARGET"].values)

# Accuracy: 81% 

In [ ]:
Counter(y_train), Counter(y_val)

# Check prediction results when test set is one specific equipment after a date

In [ ]:
# collect test set by EQ
# predict aug results for that EQ
test_set = new_training[(new_training["TIMESTAMP"]>=aug) &
                        (new_training["EQUIPMENT"]=="WBA127")].drop(["TIMESTAMP", "EQUIPMENT"], axis=1)
training_set = new_training[~((new_training["TIMESTAMP"]>=aug) &
                        (new_training["EQUIPMENT"]=="WBA127"))].drop(["TIMESTAMP", "EQUIPMENT"], axis=1)

X_train = training_set.drop(["TARGET"], axis=1).values
y_train = np.array(training_set["TARGET"].values)

X_val = test_set.drop(["TARGET"], axis=1).values
y_val = np.array(test_set["TARGET"].values)

In [ ]:
Counter(y_train), Counter(y_val)

# Check prediction when it is only trained on one equipment

In [ ]:
aug = datetime(2021, 8, 1)
test_set = new_training[(new_training["TIMESTAMP"]>=aug) &
                        (new_training["EQUIPMENT"]=="WBA128")].drop(["TIMESTAMP", "EQUIPMENT"], axis=1)
training_set = new_training[(new_training["TIMESTAMP"]<=aug) &
                        (new_training["EQUIPMENT"]=="WBA128")].drop(["TIMESTAMP", "EQUIPMENT"], axis=1)

X_train = training_set.drop(["TARGET"], axis=1).values
y_train = np.array(training_set["TARGET"].values)

X_val = test_set.drop(["TARGET"], axis=1).values
y_val = np.array(test_set["TARGET"].values)

In [ ]:
Counter(y_train), Counter(y_val)

In [40]:
dt_model = DecisionTreeClassifier(random_state=42) # decision tree has no n_jobs parameter
# xgb_model = XGBClassifier(random_state=42, n_jobs=-1)
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

def to_labels(y_scores, threshold):
    return (y_scores >= threshold).astype('int')

models = [rf_model]
for model in models:
    start = datetime.now()
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    y_score = model.predict_proba(X_val)[:,1]

#     end = datetime.now()
#     print(f"Training took {(end-start).seconds} seconds")
#     print(confusion_matrix(y_val, pred))
#     print(f"Prediction Accuracy for {type(model).__name__} is {accuracy_score(y_val, pred)}")
print(confusion_matrix(y_val, pred), accuracy_score(y_val, pred))
print("Model out of bag score = ", rf_model.oob_score_)

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_val, y_score)
J = tpr - fpr
ix = np.argmax(J)
best_thresh = thresholds[ix]
print('Best Threshold=%f' % (best_thresh))

testest = to_labels(y_score, best_thresh) # best thresh optimizes recall
confusion_matrix(y_val, testest), accuracy_score(y_val, testest)

[[15046   688]
 [ 3953   659]] 0.7718961958124447
Model out of bag score =  0.8942395124228945
Best Threshold=0.178988


(array([[8286, 7448],
        [ 526, 4086]]),
 0.6080802123267472)

In [ ]:
# cross validation to check if picking from the center would yield different results
from sklearn.model_selection import KFold 

k = 5
kf = KFold(n_splits=k, random_state=None) 
acc_score = []

X = new_training.drop(["TIMESTAMP", "EQUIPMENT", "TARGET"], axis=1)
y = new_training["TARGET"]

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

In [ ]:
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
pred = rf_model.predict(X_val)

print(confusion_matrix(y_val, pred))
print(f"Prediction Accuracy for {type(rf_model).__name__} is {accuracy_score(y_val, pred)}")